In [68]:
import praw
import re
import string
import pandas as pd
from collections import Counter

### Reddit API access
Once you create a Reddit app in the dev portal
retrieve the following params from it

In [2]:
APP_ID = 'ACqyZR3hYsXzfA'
APP_SECRET = 'xehsNEoOe_EH9DT-JwxekUDXLKxOIA'
USER_AGENT = 'my little scrapper'

### PRAW: Python Reddit API Wrapper
Does all the hard work
https://praw.readthedocs.io/en/latest/index.html

In [52]:
reddit = praw.Reddit(
     client_id=APP_ID,
     client_secret=APP_SECRET,
     user_agent=USER_AGENT
)

In [53]:
subreddit = reddit.subreddit("wallstreetbets")

In [54]:
hot_submissions = []

for submission in subreddit.hot(limit=10):
    author_name = submission.author.name
    author_redditor_instance = reddit.redditor(author_name) 
    new_sumbission = {
        'title': submission.title,  # Output: the submission's title
        'score': submission.score,  # Output: the submission's score
        'submission_id': submission.id,     # Output: the submission's ID
        'submission_url': submission.url,   # Output: the URL the submission points to
        'author': {
            'name': author_name,
            'comment_karma': author_redditor_instance.comment_karma
        }
    }
    hot_submissions.append(new_sumbission)

In [55]:
all_comments_raw = []

def get_comments(comments):  
    for comment in comments:
        # regular comment
        if type(comment).__name__ == "Comment":
            all_comments_raw.append(comment)
        # we reach the end of the list
        # and need to load more comments
        elif type(comment).__name__ == "MoreComments":
            # recursively call itself
            get_comments(comment.comments())
        else:
            print(comment)
    
for hot_sub in hot_submissions:
    hot_sub_url = hot_sub['submission_url']
    # some urls point to images
    # which cannot be used in the submission search
    if hot_sub_url.endswith("/"):
        sub = reddit.submission(url=hot_sub['submission_url'])
        get_comments(sub.comments)
    else:
        print(hot_sub_url)

KeyboardInterrupt: 

In [75]:
all_parsed_comments = []

for comment in all_comments_raw:
    parsed_comment = parse_comment(comment)
    if parse_comment:
        all_parsed_comments.append(parsed_comment)

In [74]:
all_comments = []


def get_author_comment_karma(author_name):
    redditor = reddit.redditor(author_name)
    comment_karma = redditor.comment_karma if redditor.comment_karma else None
    return comment_karma

def parse_comment(comment):
    if hasattr(comment, 'body'):
        author_name = comment.author.name if comment.author else None
        num_of_replies = 0
        if (len(comment.replies._comments) > 0):
            num_of_replies = comment.replies._comments[0].count if hasattr(comment.replies._comments[0], 'count') else 0
        parsed_comment = {
            'comment_id': comment.id,
            'thread': hot_sub['title'],
            'body': comment.body,
            'author': author_name,
            'ups': comment.ups,
            'downs': comment.downs,
            'awards_count': comment.total_awards_received,
            'score': comment.score,
            'created_at': comment.created,
            'num_of_replies': num_of_replies
        }
        return parsed_comment
    else:
        return None

    
def process_submission_comments(comments):  
    for comment in comments:
        # regular comment
        if type(comment).__name__ == "Comment":
            new_parsed_comment = parse_comment(comment)
            # only append in comment parsed is not null
            if new_parsed_comment:
                all_comments.append(new_parsed_comment)
        # we reach the end of the list
        # and need to load more comments
        elif type(comment).__name__ == "MoreComments":
            # recursively call itself
            process_submission_comments(comment.comments())
        else:
            print(comment)
    
for hot_sub in hot_submissions:
    hot_sub_url = hot_sub['submission_url']
    # some urls point to images
    # which cannot be used in the submission search
    if hot_sub_url.endswith("/"):
        sub = reddit.submission(url=hot_sub['submission_url'])
        process_submission_comments(sub.comments)
    else:
        print(hot_sub_url)

KeyboardInterrupt: 

In [286]:
# first version, assumes you do not know the stocks by name
# uses regex to filter comment for exaclty three capital letters in a row
# with a space or a dollar sign before and a space after
# returns too many common words, not usable right now
# TODO look into exclusion with a word library

stock_name_re = '( [A-Z]{3} | \$[A-Z]{3} )'

stock_mentions = {}

for com in all_comments:
    com_stocks_names = re.findall(stock_name_re, com['body'])
    if len(com_stocks_names) > 0:
        for com_stock_name in com_stocks_names:
            name = com_stock_name.strip()
            if name[0] == '$':
                name = name[1:]
            if name in stock_mentions:
                stock_mentions[name] += 1
            else:
                stock_mentions[name] = 1

In [46]:
# second version
# uses a list with stock names
# and Counters to count their occurencs in comments

stocks_to_look_for = ['gme', 'amc', 'nok', 'gamestop']

bullish_words = ['hold', 'buy', 'buying', 'holding', 'yolo', 'long', 'bull', 'up', 'keep', 'moon', 'go', 'launch']

bearish_words = ['sell', 'short', 'shorting', 'selling', 'bear', 'down', 'crash']

negation_list = ['why', 'no', 'not', 'don\'t']

res = {
    'GME': 0,
    'AMC': 0
}

for com in all_comments:
    text = com['body'].lower().translate(str.maketrans('', '', string.punctuation)).split()
    stock_counter = Counter(w for w in text if w in stocks_to_look_for)
    bullish_counter = Counter(w for w in text if w in bullish_words)
    bearish_counter = Counter(w for w in text if w in bearish_words)
    
    print(positive_counter, sum(positive_counter.values()))
    print(negative_counter,  sum(negative_counter.values()))
    print(stock_counter)
    print(text)
    
    print('\n')
    print('-----------------')
    print('\n')
    
    if len(stock_counter) >0:
        if 'gme' in stock_counter or 'gamestop' in stock_counter:
            res['GME'] += stock_counter['gme']
        if 'amc' in stock_counter:
            res['AMC'] += stock_counter['amc']

res

Counter() 0
Counter() 0
Counter({'gme': 1})
['gme', 'daily', 'thread', 'httpsredditcomrwallstreetbetscommentslbm3vrgmesuperawesomemegathreadweds23']


-----------------


Counter() 0
Counter() 0
Counter({'gme': 1, 'amc': 1})
['the', 'nice', 'thing', 'about', 'having', 'both', 'gme', 'and', 'amc', 'is', 'that', 'i', 'only', 'have', 'to', 'look', 'at', 'one', 'chart']


-----------------


Counter() 0
Counter() 0
Counter()
['i', 'won', 'a', '£4000', 'law', 'suit', 'recently', 'x200b', 'im', 'playing', 'with', 'somebody', 'elses', 'money', 'x200b', 'this', 'is', 'a', 'guy', 'who', 'once', 'bought', 'his', 'ex', 'girlfriend', 'a', 'christmas', 'tree', 'to', 'get', 'her', 'to', 'talk', 'to', 'him', 'again', 'x200b', 'they', 'have', 'no', 'idea', 'how', 'retarded', 'i', 'actually', 'am', 'x200b', '🚀🚀🚀🚀']


-----------------


Counter() 0
Counter() 0
Counter()
['isnt', 'it', 'weird', 'all', 'these', 'people', 'arent', 'spamming', 'about', 'silver', 'anymore']


-----------------


Counter() 0

-----------------


Counter() 0
Counter() 0
Counter({'amc': 1, 'gme': 1})
['amc', 'and', 'gme', 'are', 'doing', 'something', 'in', 'the', 'premarket', 'after', 'that', 'dark', 'yesterday']


-----------------


Counter({'buy': 2, 'hold': 2}) 4
Counter() 0
Counter({'amc': 2})
['any', 'predictions', 'for', 'amc', 'today', 'bought', '1000', 'shares', 'at', '755', 'yesterday', 'lets', 'just', 'buy', 'and', 'hold', 'and', 'see', 'what', 'happens', 'nothing', 'to', 'lose', 'if', 'you', 'put', 'in', 'money', 'u', 'prepared', 'to', 'lose', 'buy', 'hold', 'amc', 'not', 'advice']


-----------------


Counter() 0
Counter() 0
Counter()
['buyy', 'amc🚀🚀']


-----------------


Counter() 0
Counter() 0
Counter({'amc': 1})
['the', 'moon', 'lets', 'take', 'amc', 'there', 'today']


-----------------


Counter() 0
Counter() 0
Counter({'amc': 1})
['jeez', 'mods', 'make', 'an', 'amc', 'thread', 'too', 'please']


-----------------


Counter({'buy': 1}) 1
Counter() 0
Counter({'amc': 1})
['yuhuuu', 'can', '



Counter() 0
Counter() 0
Counter({'amc': 1})
['amc', 'boysss', 'i', 'just', 'love', 'movie']


-----------------


Counter({'long': 1}) 1
Counter({'down': 1}) 1
Counter({'gme': 1, 'amc': 1})
['tripled', 'down', 'this', 'morning', 'gme', 'amc', 'im', 'in', 'it', 'for', 'the', 'long', 'haul']


-----------------


Counter() 0
Counter() 0
Counter({'amc': 1})
['amc', '🚀🚀🚀', '🌔🌔🌔']


-----------------


Counter({'hold': 1}) 1
Counter() 0
Counter({'amc': 1})
['amc', 'is', 'being', 'gutted', 'hold', 'strong']


-----------------


Counter() 0
Counter() 0
Counter({'amc': 1})
['amc', 'was', 'on', 'a', 'discount', 'so', 'i', 'bought', 'another', 'share', 'lol']


-----------------


Counter() 0
Counter() 0
Counter({'nok': 2})
['nok', 'nok', 'whos', 'there', 'earnings', 'call', 'tomorrow', 'earnings', 'call', 'tomorrow', 'who', 'orange', 'you', 'glad', 'you', 'held', 'you', 'fucking', 'smooth', 'brain', 'ape']


-----------------


Counter() 0
Counter({'down': 1}) 1
Counter()
['why', 'is', 'it',

Counter({'amc': 1})
['amc', 'is', 'ready', 'to', 'takeoff', '🚀🚀🚀']


-----------------


Counter({'keep': 1, 'buying': 1}) 2
Counter() 0
Counter({'amc': 1})
['amc', 'is', 'hitting', '9', 'today', 'keep', 'buying']


-----------------


Counter() 0
Counter() 0
Counter({'gme': 1, 'amc': 1})
['first', 'investment', 'ever', 'just', 'bought', '5', 'shares', 'of', 'gme', 'and', '15', 'of', 'amc']


-----------------


Counter() 0
Counter() 0
Counter()
['nicehttpsimgurcomaydefqh2']


-----------------


Counter() 0
Counter({'down': 1}) 1
Counter()
['would', 'suggest', 'something', 'less', 'volatile', 'downvote', 'me', 'if', 'u', 'want', 'but', 'the', 'hype', 'is', 'dying', 'down']


-----------------


Counter() 0
Counter() 0
Counter()
['little', 'late', 'will', 'lose', 'money', 'now']


-----------------


Counter() 0
Counter() 0
Counter()
['sndl', 'just', 'hit', 'a', 'new', 'high']


-----------------


Counter() 0
Counter() 0
Counter()
['its', 'happening']


-----------------


Counter() 0

Counter() 0
Counter() 0
Counter()
['you', 'inspire', 'me', 'bro']


-----------------


Counter() 0
Counter() 0
Counter({'nok': 1})
['nok', 'fds', 'earnings', 'tomorrow']


-----------------


Counter() 0
Counter() 0
Counter()
['are', 'they', 'releasing', 'their', 'reports', 'tomorrow']


-----------------


Counter() 0
Counter() 0
Counter({'gme': 1, 'amc': 1})
['3', 'gme', 'and', 'another', '16', 'shares', 'of', 'amc', 'diamond', 'hands', 'guys']


-----------------


Counter() 0
Counter() 0
Counter()
['sndl', 'boys']


-----------------


Counter() 0
Counter() 0
Counter({'amc': 1})
['amc', 'volume58953556avg', 'volume107743182']


-----------------


Counter() 0
Counter() 0
Counter()
['what', 'where', 'did', 'you', 'get', 'this', 'average', 'v', 'number']


-----------------


Counter() 0
Counter() 0
Counter()
['yahoo', 'finance']


-----------------


Counter() 0
Counter() 0
Counter()
['httpsfinanceyahoocomquoteamcpamchttpsfinanceyahoocomquoteamcpamc']


-----------------


Counter(

...

Counter({'down': 1}) 1
Counter()
['you’ll', 'still', 'be', 'down', 'when', 'it', 'hits', '20']


-----------------


Counter() 0
Counter() 0
Counter()
['what', 'are', 'you', 'even', 'doing', 'in', 'this', 'thread', 'with', 'that', 'kind', 'of', 'attitude']


-----------------


Counter() 0
Counter({'down': 2}) 2
Counter()
['let’s', 'go', 'boys', 'we', 'got', 'em', 'on', 'the', 'run', 'we', 'ain’t', 'come', 'this', 'far', 'to', 'back', 'down', 'now', 'fuck', 'that', 'shit', 'we', 'ain’t', 'goin', 'home', 'so', 'sit', 'your', 'ass', 'down', 'and', 'take', 'what’s', 'comin']


-----------------


Counter() 0
Counter() 0
Counter()
['bring', 'them', 'hell']


-----------------


Counter() 0
Counter() 0
Counter()
['in', 'a', 'way', 'the', 'dip', 'yesterday', 'is', 'good', 'seperates', 'the', 'paper', 'from', 'the', 'diamond', 'now', 'we', 'can', 'get', 'serious']


-----------------


Counter({'buy': 1, 'up': 1}) 2
Counter() 0
Counter()
['more', 'diamonds', 'can', 'buy', 'up', 'the', 'papers



Counter() 0
Counter() 0
Counter()
['how', 'do', 'people', 'find', 'out', 'about', 'these', 'no', 'name', 'stocks', 'that', 'do', '100', 'in', 'a', 'day']


-----------------


Counter() 0
Counter() 0
Counter()
['throw', 'darts', 'at', 'a', 'wall', 'of', 'tickers']


-----------------


Counter() 0
Counter() 0
Counter()
['wendys', 'back', 'lot']


-----------------


Counter() 0
Counter() 0
Counter()
['vper', 'i’m', 'fully', 'invested', 'not', 'a', 'financial', 'advisor']


-----------------


Counter() 0
Counter() 0
Counter()
['they', 'don’t', 'the', 'stonks', 'find', 'you']


-----------------


Counter() 0
Counter() 0
Counter()
['🔮']


-----------------


Counter() 0
Counter() 0
Counter()
['i', 'get', 'them', 'from', 'my', 'wife’s', 'boyfriend']


-----------------


Counter({'hold': 1}) 1
Counter() 0
Counter({'gme': 1, 'amc': 1, 'nok': 1})
['hold', 'on', 'for', 'dear', 'life', 'chads', 'this', 'pit', 'stop', 'for', 'fuel', 'is', 'almost', 'over', 'gme', 'amc', 'nok', '🚀🚀🚀']


----

Counter()
['my', 'entry', 'was', '494', 'but', 'i', 'avg', 'down', 'to', '376', 'ape', 'strong']


-----------------


Counter() 0
Counter() 0
Counter()
['bruv', 'you', 'literally', 'joined', 'at', 'the', 'top', 'x200b', 'of', 'fucking', 'nothing', '2kshare', 'lesgoooooo']


-----------------


Counter({'up': 1, 'hold': 1}) 2
Counter() 0
Counter({'gme': 1, 'amc': 1})
['thankfully', 'my', 'sndl', 'gains', 'are', 'making', 'up', 'for', 'my', 'gme', 'and', 'amc', 'losses', 'so', 'i', 'can', 'hold', 'forever']


-----------------


Counter() 0
Counter() 0
Counter()
['the', 'sale', 'yesterday', 'was', 'good', '😎']


-----------------


Counter() 0
Counter({'sell': 1}) 1
Counter({'amc': 1})
['i’ve', 'lost', 'nothing', 'because', 'i', 'did', 'not', 'sell', 'amc', '15', 'baby']


-----------------


Counter({'holding': 1}) 1
Counter() 0
Counter()
['holding', '43', 'with', 'you']


-----------------


Counter() 0
Counter() 0
Counter()
['holdholdholdboom']


-----------------


Counter({'holding

['remembering', 'the', 'guy', 'who', 'drank', 'his', 'own', 'piss', 'with', 'tears', 'in', 'my', 'eyes']


-----------------


Counter() 0
Counter() 0
Counter()
['mom', 'found', 'the', 'piss', 'stonks']


-----------------


Counter() 0
Counter() 0
Counter()
['bro', 'all', 'i', 'want', 'is', 'roblox', 'to', 'ipo']


-----------------


Counter() 0
Counter() 0
Counter()
['hodlllll']


-----------------


Counter() 0
Counter() 0
Counter()
['from', 'hero', 'to', 'villain', 'the', 'story', 'of', 'robinhood']


-----------------


Counter({'buy': 1}) 1
Counter() 0
Counter()
['twist', 'since', 'rh', 'and', 'shitadel', 'are', 'in', 'cahoots', 'vlad', 'would', 'have', 'know', 'about', 'upcoming', 'shenanigans', 'he', 'really', 'was', 'trying', 'to', 'protect', 'the', 'people', 'dont', 'buy', 'these', 'theyre', 'going', 'to', 'tank', 'the', 'price']


-----------------


Counter() 0
Counter() 0
Counter()
['pushing', 'little', 'children', 'with', 'their', 'fully', 'automatics', 'they', 'like', '

{'GME': 659, 'AMC': 1151}

In [93]:
df = pd.DataFrame.from_dict(all_parsed_comments)
# df.created_at = pd.to_datetime(df.created_at)

In [98]:
df.sort_values(by=['score'])

,comment_id,thread,body,author,ups,downs,awards_count,score,created_at,num_of_replies
1348,glwsi4v,"What Are Your Moves Tomorrow, February 04, 2021",No save your money.,I-Have-TheHigh-Groud,-66,0,0,-66,1.612416e+09,0
1376,glynyfm,"What Are Your Moves Tomorrow, February 04, 2021",I remember when GME getting to 420.69 was a me...,Complex_Equipment958,-30,0,0,-30,1.612451e+09,0
4743,glx2vgu,"What Are Your Moves Tomorrow, February 04, 2021",Fuck off loser,CoffeeAndClen,-24,0,0,-24,1.612420e+09,0
3230,gly3jmr,"What Are Your Moves Tomorrow, February 04, 2021",Shutup,Marty9822,-21,0,0,-21,1.612438e+09,0
8537,glwtfhx,"What Are Your Moves Tomorrow, February 04, 2021",BAN,churnAmex,-16,0,0,-16,1.612416e+09,0
...,...,...,...,...,...,...,...,...,...,...
22,glwpvwu,"What Are Your Moves Tomorrow, February 04, 2021",# WHO'S HOLDING GME 💎🙌,JiggazInParis,1726,0,1,1726,1.612415e+09,0
1,glx3bm3,"What Are Your Moves Tomorrow, February 04, 2021","Hey, SEC, I’m manipulating the market and I’m ...",real-fuzzy-dunlop,1810,0,4,1810,1.612421e+09,0
3,glwpmhs,"What Are Your Moves Tomorrow, February 04, 2021",We closed green. WE HOLD AND BUY TOMORROW,TouchMyWillyy,2216,0,1,2216,1.612415e+09,0
8,glwpn3t,"What Are Your Moves Tomorrow, February 04, 2021",Fuck the SEC,bojangles522,2263,0,8,2263,1.612415e+09,27


In [91]:
pd.to_datetime(df.created_at)

0       1970-01-01 00:00:01.612419271
1       1970-01-01 00:00:01.612420672
2       1970-01-01 00:00:01.612420431
3       1970-01-01 00:00:01.612414840
4       1970-01-01 00:00:01.612450418
                     ...             
10818   1970-01-01 00:00:01.612416704
10819   1970-01-01 00:00:01.612417348
10820   1970-01-01 00:00:01.612421150
10821   1970-01-01 00:00:01.612417357
10822   1970-01-01 00:00:01.612417422
Name: created_at, Length: 10823, dtype: datetime64[ns]

In [104]:
import datetime
pd.to_datetime(datetime.datetime.fromtimestamp(df.created_at[0]))

Timestamp('2021-02-04 07:14:31')

In [106]:
pd.Timestamp(df.created_at[0])

Timestamp('1970-01-01 00:00:01.612419271')